In [1]:
% load_ext autoreload
% autoreload 2

In [2]:
import utils
import train
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

In [4]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 20

num_epochs=10

anneal = utils.kl_anneal_linear

eos = EN.vocab.stoi["</s>"]
bos = EN.vocab.stoi["<s>"]
pad = EN.vocab.stoi["<pad>"]

filter_beam = [pad, bos]

In [6]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()

/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [7]:
train.train(model,model_name, train_iter, val_iter, DE, EN, anneal, num_epochs, gpu)

  0%|          | 0/3722 [00:00<?, ?it/s]/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
 18%|█▊        | 664/3722 [01:35<07:20,  6.94it/s]

KeyboardInterrupt: 

In [15]:
import beam_search
import torch
k = 10
bos = EN.vocab.stoi['<s>']
eos = EN.vocab.stoi['</s>']
max_len = 20
    
for batch in train_iter:
    if gpu:
        src = batch.src.cuda()
    else:
        src = batch.src
    break

src = src[:,:1]

In [16]:
best_options = beam_search.beam_search(model, src, k, max_len, bos, eos, filter_token=filter_beam, gpu=gpu)    

/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [17]:
for lprob, sentence, _ in best_options:
    sent = ""
    for word in sentence:
        sent += EN.vocab.itos[word] + " "
    print(sent)

I And I 'm 
I I 'm not 


In [12]:
sent = ""
for word in src:
    sent += DE.vocab.itos[word] + " "
print(sent)

Lassen Sie uns sehen , ob wir die dritte Partei hier anwenden können . 


In [24]:
sentences = utils.generate(model, val_iter, EN, k, max_len, gpu)


  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 18/18 [14:43<00

In [35]:
b, r = utils.test_generation(model, val_iter, EN, k, max_len, gpu)

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
100%|██████████| 18/18 [02:51<00:00,  9.54s/it]


TypeError: sequence item 0: expected str instance, Tensor found

In [ ]:
len(sentences)